In [11]:
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.append('./../../code/')
from dn_utils.behavioral_models import (load_behavioral_data, 
    estimate_values_pd, estimate_pred_err)

### Observation
Usually subjects are trying to minimize overal prediction error. Here it is not precisely the case, because sometimes subject pick probabilistically worse option to maximize their winnings.

In [18]:
# Load data
beh_path = '/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/' \
           'data/main_fmri_study/sourcedata/behavioral'
param_path = '/home/kmb/Desktop/Neuroscience/Projects/BONNA_decide_net/' \
             'code/behavioral_analysis/matjags-dn/data/alpha_pd_mle.npy'

beh, meta = load_behavioral_data(beh_path)
alpha_pd_mle = np.load(param_path)

n_subjects = len(meta['dim1'])
n_conditions = len(meta['dim2'])
n_trials = len(meta['dim3'])

Shape of beh array: (32, 2, 110, 21)
Conditions [(0, 'rew'), (1, 'pun')]
Columns: [(0, 'block'), (1, 'rwd'), (2, 'magn_left'), (3, 'magn_right'), (4, 'response'), (5, 'rt'), (6, 'won_bool'), (7, 'won_magn'), (8, 'acc_after_trial'), (9, 'onset_iti'), (10, 'onset_iti_plan'), (11, 'onset_iti_glob'), (12, 'onset_dec'), (13, 'onset_dec_plan'), (14, 'onset_dec_glob'), (15, 'onset_isi'), (16, 'onset_isi_plan'), (17, 'onset_isi_glob'), (18, 'onset_out'), (19, 'onset_out_plan'), (20, 'onset_out_glob')]


### Calculate and save PE regressors.

In [27]:
pe_regressors = np.zeros((n_subjects, n_conditions, n_trials))

for sub in range(n_subject):
    
    for con in range(n_conditions):
        
        val = estimate_values_pd(
            beh, meta, sub, con,
            alpha_plus=alpha_pd_mle[sub, con, 0], 
            alpha_minus=alpha_pd_mle[sub, con, 1]
        )
        pe_regressors[sub, con, :] = estimate_pred_err(
            beh, meta, sub, con, val
        )
        
np.save('data/pe_regressors', pe_regressors)